In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Definamos el sistema en una tupla
G=(lambda x,y,z: 3*x - np.cos(y*z) - 1.5, \
   lambda x,y,z: 4*x**2 - 625*y**2 + 2*y -1, \
   lambda x,y,z: np.exp(-x*y) + 20*z + 9.471975 )

In [3]:
for i in range(3):
    print(G[i](0,0,0))

-2.5
-1
10.471975
